Query Historical Weather Data<br>
visualcrossing.com

In [1]:
import pandas as pd
import requests
import datetime

In [3]:
# Function to get weather data from Visual Crossing API
def get_visual_crossing_historical_weather(api_key, location, start_date, end_date):
    url = f"https://weather.visualcrossing.com/VisualCrossingWebServices/rest/services/timeline/{location}/{start_date}/{end_date}"
    
    params = {
        'unitGroup': 'us',  # 'metric' for Celsius, 'us' for Fahrenheit
        'key': api_key,
        'include': 'days',  # This includes daily summary
    }
    
    response = requests.get(url, params=params)
    
    if response.status_code == 200:
        data = response.json()
        return data
    else:
        print(f"Error: {response.status_code}, {response.text}")
        return None

# Read the CSV file
track_accidents = pd.read_csv('track_accidents_last_10_years.csv')

# Initialize your API key
api_key = 'AEMGQUX9PJLNHQYP3ZXVLG9VQ'  # Replace with your Visual Crossing API key

# Define the range of incidents to process
start_index = 4500   # Update this for each day's run
stop_index = 4799 # Update this for each day's run

# Extract the relevant subset of incidents
incident_subset = track_accidents.iloc[start_index:stop_index]

# List to hold weather data for each incident
weather_data = []

# Iterate through each row in the subset
for index, row in incident_subset.iterrows():
    station = row['station']
    stateabbr = row['stateabbr']
    incident_date = row['date']

    # Convert the incident date to a datetime object
    incident_datetime = datetime.datetime.strptime(incident_date, '%Y-%m-%d')
    
    # Calculate the start and end dates for the weather query
    start_date = (incident_datetime - datetime.timedelta(days=1)).strftime('%Y-%m-%d')  # Day prior
    end_date = (incident_datetime + datetime.timedelta(days=1)).strftime('%Y-%m-%d')    # Following day
    
    # Construct the location as 'City,State'
    location = f"{station},{stateabbr}"
    
    # Get weather data for the specified date range
    weather = get_visual_crossing_historical_weather(api_key, location, start_date, end_date)
    
    # Store the results if available
    if weather:
        # Initialize temperatures
        prior_temp = actual_temp = following_temp = 'N/A'
        
        for day in weather['days']:
            # Assign temperatures based on the day
            if day['datetime'] == (incident_datetime - datetime.timedelta(days=1)).strftime('%Y-%m-%d'):
                prior_temp = day.get('temp', 'N/A')
            elif day['datetime'] == incident_datetime.strftime('%Y-%m-%d'):
                actual_temp = day.get('temp', 'N/A')
            elif day['datetime'] == (incident_datetime + datetime.timedelta(days=1)).strftime('%Y-%m-%d'):
                following_temp = day.get('temp', 'N/A')

        # Append the data along with original incident information
        weather_data.append({
            **row.to_dict(),  # Add original incident data
            'prior_temp': prior_temp,
            'actual_temp': actual_temp,
            'following_temp': following_temp,
        })

# Convert the results into a DataFrame for further analysis
weather_df = pd.DataFrame(weather_data)

# Save to CSV file, including the range in the filename
output_filename = f'incident_weather_data_{start_index}_{stop_index}.csv'
weather_df.to_csv(output_filename, index=False)

print(f"Processed and saved weather data for incidents {start_index} to {stop_index} into {output_filename}")

Processed and saved weather data for incidents 4500 to 4799 into incident_weather_data_4500_4799.csv
